## Initialisering
For at eksemplene i notebooken skal fungere smertefritt, må openai og dotenv (python-dotenv)-bibliotekene være installert i miljøet der du starter notebooken. Om nødvendig kan du kjøre `!pip install <library>` i en kode-celle for å installere et manglende bibliotek. 

Før vi begynner å gjøre noe interessant, må vi importere noen bibliotek og metoder, og vi må lese inn en gyldig openai-nøkkel (leses her fra en lokal .env - fil).

In [ ]:
# bibliotek vi trenger i dette kapittelet
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import numpy as np
import tiktoken
import json
from pprint import pprint

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from IPython.display import display, HTML

# nyttig pandas setting
pd.options.mode.copy_on_write = True

In [ ]:
# leser API-nøkkel og initialiserer OpenAI-klient

_ = load_dotenv(find_dotenv()) # leser fra lokal .env - fil
openai.api_key  = os.environ['OPENAI_API_KEY']

client=OpenAI()

## Gruppering og klassifisering av tekst ved hjelp av embeddings

Siden embeddings jo bare er vektorer med flyttall, er de teknisk sett enkle å plugge inn i forskjellige analysemetoder. En interessant mulighet er å  bruke ulike dimensjonsreduserings-teknikker til å gruppere og klassifisere tekster. 

Under gjør vi en prinsipalkomponent - analyse av eksempeldataene vi brukte over. Litt forenklet "tvinger" vi det 1536-dimensjonale rommet vi egentlig er i ned i færre dimensjoner, som er enklere for menneskehjerner å forholde seg til. 

Før vi begynner definerer vi noen hjelpefunksjoner:

In [ ]:
# Lager nye hjelpefunksjoner for mer effektiv håndtering av embeddings

# hjelpefunksjon for å sjekke antall tokens i tekst
def token_count(text, encoding_model):
    encoding=tiktoken.get_encoding(encoding_model)
    n_tokens = len(encoding.encode(text))
    return n_tokens
    
# funksjon som for en gitt tekst (text) returnerer embedding-vektor som numpy-array 
def embed_helper_2(AIclient, text, model_name):
    embedding=client.embeddings.create(
          model=model_name,
          input=text,
          encoding_format="float")
    return np.array(embedding.data[0].embedding).reshape(1,-1)

# hjelpefunksjon for å lagre embeddings, lables og eventuelle kategorier (cats) fra dataframe i numpy arrays.
def embed_df(AIclient, df, text_col, label_col, cat_col, model_name): 
    embedding_array=df.apply(lambda row: embed_helper_2(AIclient, row[text_col], model_name)[0], axis=1, result_type="expand")
    labels=df[label_col]
    cats=df[cat_col]
    return np.asarray(embedding_array), np.asarray(labels),  np.asarray(cats)

# similaritetsberegning for array
def similarity_rank_2(AIclient, embed_array, input_text, embed_model): 
    input_text_embedded=embed_helper_2(AIclient, input_text, embed_model)
    similarities=cosine_similarity(embed_array,input_text_embedded)
    return similarities


In [ ]:
# prinsipalkomponent-analyse m plotting av 2 første komponenter

def plot_embeddings(embedding_vectors, labels, cats):
    # Create a PCA model
    pca_model = PCA(random_state=42)

    # Fit and transform the data to obtain PCA coordinates
    pca_result = pca_model.fit(embedding_vectors)
    
    #print("PCA 2-components explained variance:" + str(pca_result.explained_variance_ratio_) + "\n")
    pca_trans = pca_model.fit_transform(embedding_vectors)

    # Plot the PCA results
    # explained variance
    fig=plt.figure(figsize=(8, 12))
    
    ax=fig.add_subplot(211)
    plt.title('PCA Explained variance by no. of components')
    plt.plot(np.cumsum(pca_result.explained_variance_ratio_))
    plt.grid(alpha=0.2)

     # scatterplot by artist
    ax=fig.add_subplot(212)
    plt.title('PCA Projection of Embedding Vectors')
    
    cat_list=list(set(cats))
    cat_vals=np.asarray([cat_list.index(c) for c in cats])

    cmap='tab20'
    color_map = plt.colormaps[cmap].resampled(20)
      
    for i, cat in enumerate(cat_list):
        filter_arr = []
        for catval in cats:
            if catval==cat:
                filter_arr.append(True)
            else:
                filter_arr.append(False)

        plt.scatter(pca_trans[filter_arr, 0], pca_trans[filter_arr, 1], color=color_map(i/len(cat_list)), label=cat, s=20)  

    plt.legend()
    plt.grid(alpha=0.2)
    plt.xlabel('PCA component 1')
    plt.ylabel('PCA component 2')
    plt.show()       


Filen "lyrics.csv" inneholder rundt 25 000 sangtekster med kjente, kjære og mindre kjære artister. Under tester vi noen av de samme teknikkene på disse dataene. 

Først leser vi inn hele datasettet og lager etpar hjelpevariable. 

In [ ]:
# innlesning av sangtekst datasett (25 000 sanger)
df_sang = pd.read_csv('data/lyrics.csv', header=0, sep=',')

# hjelpevariable for litt enklere oppslag og navigasjon
df_sang['n_tokens']=df_sang.apply(lambda row: token_count(row['lyrics'], "cl100k_base"), axis=1)
df_sang['song_id']=df_sang.index
df_sang['artist']=df_sang['artist'].apply(lambda x: x[:-7]) # fjerner unødvendig "Lyrics" i artistnavn 
df_sang['song_label']=df_sang['song_id'].apply(str) + " - " + df_sang['artist'] + " - " +df_sang['song_name']

# rask inspeksjon
df_sang

In [ ]:
# sjekk av artister - viser de 200 med flest sanger (kan justeres)
pd.options.display.max_rows=200
df_sang.value_counts('artist', ascending=False)[:200]

In [ ]:
# sjekk sanger for en valgt artist
df_sang[df_sang['artist']=='Snoop Dogg'][:200]['song_label']

For å begrense ventetid lager vi et tilfeldig sample på 100 sanger (kan justeres) som vi bruker videre. I koden under er samplet begrenset til noen utvalgte artister - gjør gjerne endringer i utvalget!

In [ ]:
# lager sample på 100 sanger, noen valgte artister
df_sang_sample=df_sang[df_sang['artist'].isin(['Bob Dylan', 'Snoop Dogg', 'Backstreet Boys', 'Lana Del Rey', 'Eminem', 'Taylor Swift'])].sample(n=100).reset_index()
print(df_sang_sample['song_label'][:100])

Vi lager så embeddings av tekstene i utvalget vårt med hjelp av funksjonene vi lagde i forrige avsnitt.

In [ ]:
# lager array med embeddings
sang_embed_array, sang_embed_lbl, sang_embed_cat=embed_df(client, df_sang_sample, 'lyrics', 'song_label', 'artist', "text-embedding-3-small")

Vi kan nå sjekke similaritet mellom tekstene, eller med annen input - tekst om vi ønsker:

In [ ]:
# beregning av similaritet med input-tekst

#ref_lyrics: sangteksten vi ønsker å sammenligne de andre med. Her kan man selvfølgelig også oppgi en annen, vilkårlig tekst 
ref_lyrics=df_sang_sample.iloc[1]["lyrics"]

similarities=similarity_rank_2(client, sang_embed_array, ref_lyrics, 'text-embedding-3-small')
df_sang_sample["similarities_arr"]=similarities

In [ ]:
df_sang_sample

In [ ]:
# viser topp n og laveste n tekster, rangert etter similaritet med input_tekst
n=5
sim_col="similarities_arr"

values_topn=list(df_sang_sample.nlargest(n,sim_col).sort_values(sim_col, ascending=True)[sim_col]) 
values_smalln=list(df_sang_sample.nsmallest(n,sim_col).sort_values(sim_col, ascending=True)[sim_col])

names_topn=list(df_sang_sample.nlargest(n,sim_col).sort_values(sim_col, ascending=True)["song_label"])
names_smalln=list(df_sang_sample.nsmallest(n,sim_col).sort_values(sim_col, ascending=True)["song_label"])

values=values_smalln+values_topn
names=[c.replace('$', 'S') for c in names_smalln+names_topn] # Paid da cost to be da bo$$ quick fix  

fig, ax = plt.subplots(figsize=(10,5))
ax.barh(names_smalln, values_smalln)
ax.barh(names_topn, values_topn)

plt.title('Top/bottom similarity with ' + ref_lyrics[:50] + '...')
plt.show()


Er det noen fornuft i rangeringen? Bruk gjerne hjelpefunksjonen under for å inspisere to tekster ved siden av hverandre. 

In [ ]:
# hjelpefunksjon for sjekk av tekster mot hverandre
def view_lyrics(df, song_id_1, song_id_2):
    df_temp=df_sang_sample.loc[df_sang_sample['song_id'] == song_id_1].reset_index()
    label1=df_temp.loc[0, "song_label"]
    lyrics1=df_temp.loc[0, "lyrics"]

    df_temp=df_sang_sample.loc[df_sang_sample['song_id'] == song_id_2].reset_index()
    label2=df_temp.loc[0, "song_label"]
    lyrics2=df_temp.loc[0, "lyrics"]
    
    html_code = f"""
    <div style="display: flex; justify-content: space-between;">
        <div style="flex: 1; padding-right: 8px;">
            <h2>{label1}</h2>
            <pre style="font-size: 9px;"> {lyrics1} </pre>
        </div>
        <div style="flex: 1; padding-left: px;">
            <h2>{label2}</h2>
            <pre style="font-size: 9px;">{lyrics2}</pre>
        </div>
    </div>
    """
    return html_code

In [ ]:
df_sang_sample

In [ ]:
# vis to tekster ved siden av hverandre - bruk sang_id for valg av tekster
# NB - gir feil hvis IDer ikke finnes i df_sang_sample dataframe, pass på å velge IDer som er i aktuelt sample.
display(HTML(view_lyrics(df_sang_sample, 10347, 726)))

Vi kan også sjekke hvordan PCA eller t-SNE (t-distributed stochastic neighbor embedding) dimensjonsreduksjon plasserer sangene i to dimensjoner. Ser det f eks ut til at sangene fra samme artist er samlet i noen grad?

For TSNE, test gjerne effekten av å justere på perplexity - parameteren.

In [ ]:
# PCA-plott av embeddings, med artist som kategori
 
plot_embeddings(sang_embed_array, sang_embed_lbl, sang_embed_cat)

In [ ]:
# t-SNE - alternativ (ikke-lineær) metode for dimensjonsreduksjon

def plot_embeddings_tsne(embedding_vectors, labels, cats, show_labels=False, perplexity=10):
    # Create a TSNE model
    tsne_model = TSNE(n_components=2, random_state=42, perplexity=perplexity)

    # Fit and transform the data to obtain PCA coordinates
    tsne_result = tsne_model.fit_transform(embedding_vectors)

    # Plot the TSNE result
    cat_list=list(set(cats))
    cat_vals=[cat_list.index(c) for c in cats]
    fig=plt.figure(figsize=(8, 6))
    ax=fig.add_subplot(111)

    plt.title('PCA Projection of Embedding Vectors')
    
    cmap='tab20'
    color_map = plt.colormaps[cmap].resampled(20)
    
    # scatterplot by artist
    for i, cat in enumerate(cat_list):
        filter_arr = []
        for catval in cats:
            if catval==cat:
                filter_arr.append(True)
            else:
                filter_arr.append(False)

        plt.scatter(tsne_result[filter_arr, 0], tsne_result[filter_arr, 1], color=color_map(i/len(cat_list)), label=cat, s=20) 
         
    plt.legend()  
    plt.title('TSNE Projection of Embedding Vectors')    
    plt.grid(alpha=0.2)
     
    plt.show()    
    
# test gjerne forskjellige verdier for perplexity
plot_embeddings_tsne(sang_embed_array, sang_embed_lbl, sang_embed_cat, perplexity=30, show_labels=False)